### How to use few shot examples in chat models
- this guide convers how to prompt  a chat  modle with  example  input and output
- this is a sampole  yet powewrfull way  to guide generation  and in some cases drastically imporve  
- there  dows not appear to be solid  consensus  on  how best  to dod few-shot

In [2]:
import  os 
from dotenv import load_dotenv

# environment variables from .env
load_dotenv("../.env")  
# Set environmental variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### fixed examples  
-  the most  basic  (and common) few-shot prompting techique is and aboid worrying about addtitonal  moveing  parts in production

`examples`  : list of dictionary examples to include  itn the  final  prmpt.
`example_prompt`  : converts each  example into 1 or more messages throught  its  
`format_messages` : method a common  example would be to convert each example  into one human  message  and on IA messagerespinse, or a human  ffollowed  bu a functino  call message 

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

model.invoke("What is 2 🦜 9?").content

'The expression "2 🦜 9" seems to use a parrot emoji (🦜) in place of a mathematical operator. If you could clarify what operation you intend to represent with the parrot emoji, I would be happy to help you solve it!'

- new let's see what haappend if we give the LLM some examples  to work with. we'll defien some below

In [5]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

In [7]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.invoke({}).to_messages())

[HumanMessage(content='2 🦜 2', additional_kwargs={}, response_metadata={}), AIMessage(content='4', additional_kwargs={}, response_metadata={}), HumanMessage(content='2 🦜 3', additional_kwargs={}, response_metadata={}), AIMessage(content='5', additional_kwargs={}, response_metadata={})]


In [8]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [10]:
from langchain_openai import ChatOpenAI

chain = final_prompt | model

chain.invoke({"input": "What is 2 🦜 9?"})

AIMessage(content='The operation represented by the parrot emoji (🦜) is not defined in standard mathematics. If you could clarify what operation you intend for the parrot to represent, I would be happy to help!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 60, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None}, id='run-a3fa1bb7-e92a-4369-85f5-1ad3c05c6fa8-0', usage_metadata={'input_tokens': 60, 'output_tokens': 42, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})